In [1]:
!pip install huggingface_hub python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 9.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [huggingface_hub] [huggingface_hub]


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env file

HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

print("Token loaded:", HF_TOKEN is not None)

Token loaded: True


In [3]:
from huggingface_hub import login
login(token=HF_TOKEN)

In [4]:
import os
os.chdir("RepText")

In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 102.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 76.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 68.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 137.2 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 109.5 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 15.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 113.6 MB/s  0:00:00m0:00:01
  Attempting uninstall: pyjwtm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 76.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [diffusers]/3 [diffusers]


In [7]:
!nvidia-smi

Sat Feb  7 08:32:23 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:D2:00.0 Off |                    0 |
|  0%   31C    P8             34W /  300W |       0MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
import torch
from controlnet_flux import FluxControlNetModel
from pipeline_flux_controlnet import FluxControlNetPipeline

from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
import re
import os

def contains_chinese(text):
    if re.search(r'[\u4e00-\u9fff]', text):
        return True
    return False

def canny(img):
    low_threshold = 50
    high_threshold = 100
    img = cv2.Canny(img, low_threshold, high_threshold)
    img = img[:, :, None]
    img = 255 - np.concatenate([img, img, img], axis=2)
    return img

In [9]:
base_model = "black-forest-labs/FLUX.1-dev"
controlnet_model = "Shakker-Labs/RepText"

controlnet = FluxControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.bfloat16)
pipe = FluxControlNetPipeline.from_pretrained(
    base_model, controlnet=controlnet, torch_dtype=torch.bfloat16
).to("cuda")

## set resolution
width, height = 1024, 1024

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/diffusers/models/embeddings.py:2617: FutureWarning: `FluxPosEmbed` is deprecated and will be removed in version 1.0.0. Importing and using `FluxPosEmbed` from `diffusers.models.embeddings` is deprecated. Please import it from `diffusers.models.transformers.transformer_flux`.
  deprecate("FluxPosEmbed", "1.0.0", deprecation_message)


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/219 [00:00<?, ?it/s]

In [ ]:


def predict(text=["hi!!"],prompt='a street sign in city',num_inference_steps=30,controlnet_conditioning_step=30,image_size=[1024,1024],file_name='result'):
    ## set text content, position, color
    
    ## set resolution
    width, height = image_size[0], image_size[1]

    ## set font
    font_path = "./assets/Arial_Unicode.ttf" # use your own font
    font_size = 80 # it is recommended to use a font size >= 60
    font = ImageFont.truetype(font_path, font_size)

    ## set text content, position, color
    text_list = text
    text_position_list = [(370, 200)]
    text_color_list = [(255, 255, 255)]

    # text_list = ["Shakker Labs"]
    # text_position_list = [(270, 300)]
    # text_color_list = [(255, 255, 255)]

    # text_list = ["Lovart AI", "Always Day 1"]
    # text_position_list = [(470, 300), (470, 400)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    # text_list = ["以往不谏", "来者可追"]
    # text_position_list = [(200, 200), (200, 300)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    # text_list = ["Shakker Labs", "RepText"]
    # text_position_list = [(200, 200), (200, 300)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    ## set controlnet conditions
    control_image_list = [] # canny list
    control_position_list = [] # position list
    control_mask_list = [] # regional mask list
    control_glyph_all = np.zeros([height, width, 3], dtype=np.uint8) # all glyphs
    
    ## handle each line of text
    for text, text_position, text_color in zip(text_list, text_position_list, text_color_list):

        ### glyph image, render text to black background
        control_image_glyph = Image.new("RGB", (width, height), (0, 0, 0))
        draw = ImageDraw.Draw(control_image_glyph)
        draw.text(text_position, text, font=font, fill=text_color)

        ### get bbox
        bbox = draw.textbbox(text_position, text, font=font)

        ### position condition
        control_position = np.zeros([height, width], dtype=np.uint8)
        control_position[bbox[1]:bbox[3], bbox[0]:bbox[2]] = 255
        control_position = Image.fromarray(control_position.astype(np.uint8))
        control_position_list.append(control_position)

        ### regional mask
        control_mask_np = np.zeros([height, width], dtype=np.uint8)
        control_mask_np[bbox[1]-5:bbox[3]+5, bbox[0]-5:bbox[2]+5] = 255
        control_mask = Image.fromarray(control_mask_np.astype(np.uint8))
        control_mask_list.append(control_mask)

        ### accumulate glyph
        control_glyph = np.array(control_image_glyph)
        control_glyph_all += control_glyph

        ### canny condition
        control_image = canny(cv2.cvtColor(np.array(control_image_glyph), cv2.COLOR_RGB2BGR))
        control_image = Image.fromarray(cv2.cvtColor(control_image, cv2.COLOR_BGR2RGB))
        control_image_list.append(control_image)
        
    control_glyph_all = Image.fromarray(control_glyph_all.astype(np.uint8))
    control_glyph_all = control_glyph_all.convert("RGB")
    # control_glyph_all.save("./results/control_glyph.jpg")

    # it is recommended to use words such 'sign', 'billboard', 'banner' in your prompt
    # for Englith text, it helps if you add the text to the prompt
    for text in text_list:
        if not contains_chinese(text):
            prompt += f", '{text}'"
    prompt += ", filmfotos, film grain, reversal film photography" # optional
    print(prompt)

    generator = torch.Generator(device="cuda").manual_seed(42)

    image = pipe(
        prompt,
        control_image=control_image_list, # canny
        control_position=control_position_list, # position
        control_mask=control_mask_list, # regional mask
        control_glyph=control_glyph_all, # as init latent, optional, set to None if not used
        controlnet_conditioning_scale=1.0,
        controlnet_conditioning_step=controlnet_conditioning_step,
        width=width,
        height=height,
        num_inference_steps=num_inference_steps,
        guidance_scale=3.5,
        generator=generator,
    ).images[0]

    if not os.path.exists("./results"):
        os.makedirs("./results")
    image.save(f"./results/{file_name}.jpg")

In [ ]:
predict()

In [ ]:
predict(text=['السلام عليكم ورحمة الله وبركاته'],file_name='I')

In [ ]:
predict(text=['السلام عليكم ورحمة الله وبركاته'],num_inference_steps=40,controlnet_conditioning_step=50,image_size=[1024,512],file_name='I2')


In [ ]:
predict(text=['السلام عليكم ورحمة الله وبركاته'],num_inference_steps=40,controlnet_conditioning_step=20,image_size=[512,512],file_name='I3')


In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=[ 'ورحمة الله وبركاته','السلام عليكم'],num_inference_steps=40,controlnet_conditioning_step=50,image_size=[1024,512],file_name='I4')


In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=['السلام عليكم ورحمة الله وبركاته'],num_inference_steps=40,controlnet_conditioning_step=50,image_size=[1024,512],file_name='I5')

In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=['السلام عليكم'],num_inference_steps=30,controlnet_conditioning_step=50,image_size=[1024,512],file_name='I6')


In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=['صباح الخير'],num_inference_steps=30,controlnet_conditioning_step=40,image_size=[1024,512],file_name='Good morning')


In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=['مرحبا بك'],num_inference_steps=30,controlnet_conditioning_step=30,image_size=[1024,512],file_name='Welcome to you_CC30')


In [ ]:
predict(prompt='A clearly visible street sign placed in the middle of a city road.',text=['مرحبا بك'],num_inference_steps=30,controlnet_conditioning_step=10,image_size=[1024,512],file_name='Welcome to you_cc10')


In [10]:
def predict_CC_loop(text=["hi!!"],prompt='a street sign in city',num_inference_steps=20,controlnet_conditioning_step_list=[10,15,20,25,30,35,40],image_size=[1024,1024],file_name='result'):
    ## set text content, position, color
    
    ## set resolution
    width, height = image_size[0], image_size[1]

    ## set font
    font_path = "./assets/Arial_Unicode.ttf" # use your own font
    font_size = 80 # it is recommended to use a font size >= 60
    font = ImageFont.truetype(font_path, font_size)

    ## set text content, position, color
    text_list = text
    text_position_list = [(370, 200)]
    text_color_list = [(255, 255, 255)]

    # text_list = ["Shakker Labs"]
    # text_position_list = [(270, 300)]
    # text_color_list = [(255, 255, 255)]

    # text_list = ["Lovart AI", "Always Day 1"]
    # text_position_list = [(470, 300), (470, 400)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    # text_list = ["以往不谏", "来者可追"]
    # text_position_list = [(200, 200), (200, 300)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    # text_list = ["Shakker Labs", "RepText"]
    # text_position_list = [(200, 200), (200, 300)]
    # text_color_list = [(255, 255, 255), (255, 255, 255)]

    ## set controlnet conditions
    control_image_list = [] # canny list
    control_position_list = [] # position list
    control_mask_list = [] # regional mask list
    control_glyph_all = np.zeros([height, width, 3], dtype=np.uint8) # all glyphs
    
    ## handle each line of text
    for text, text_position, text_color in zip(text_list, text_position_list, text_color_list):

        ### glyph image, render text to black background
        control_image_glyph = Image.new("RGB", (width, height), (0, 0, 0))
        draw = ImageDraw.Draw(control_image_glyph)
        draw.text(text_position, text, font=font, fill=text_color)

        ### get bbox
        bbox = draw.textbbox(text_position, text, font=font)

        ### position condition
        control_position = np.zeros([height, width], dtype=np.uint8)
        control_position[bbox[1]:bbox[3], bbox[0]:bbox[2]] = 255
        control_position = Image.fromarray(control_position.astype(np.uint8))
        control_position_list.append(control_position)

        ### regional mask
        control_mask_np = np.zeros([height, width], dtype=np.uint8)
        control_mask_np[bbox[1]-5:bbox[3]+5, bbox[0]-5:bbox[2]+5] = 255
        control_mask = Image.fromarray(control_mask_np.astype(np.uint8))
        control_mask_list.append(control_mask)

        ### accumulate glyph
        control_glyph = np.array(control_image_glyph)
        control_glyph_all += control_glyph

        ### canny condition
        control_image = canny(cv2.cvtColor(np.array(control_image_glyph), cv2.COLOR_RGB2BGR))
        control_image = Image.fromarray(cv2.cvtColor(control_image, cv2.COLOR_BGR2RGB))
        control_image_list.append(control_image)
        
    control_glyph_all = Image.fromarray(control_glyph_all.astype(np.uint8))
    control_glyph_all = control_glyph_all.convert("RGB")
    # control_glyph_all.save("./results/control_glyph.jpg")

    # it is recommended to use words such 'sign', 'billboard', 'banner' in your prompt
    # for Englith text, it helps if you add the text to the prompt
    for text in text_list:
        if not contains_chinese(text):
            prompt += f", '{text}'"
    prompt += ", filmfotos, film grain, reversal film photography" # optional
    print(prompt)

    generator = torch.Generator(device="cuda").manual_seed(42)
    if not os.path.exists("./results"):
        os.makedirs("./results")
    for i in controlnet_conditioning_step_list:
        image = pipe(
            prompt,
            control_image=control_image_list, # canny
            control_position=control_position_list, # position
            control_mask=control_mask_list, # regional mask
            control_glyph=control_glyph_all, # as init latent, optional, set to None if not used
            controlnet_conditioning_scale=1.0,
            controlnet_conditioning_step=i,
            width=width,
            height=height,
            num_inference_steps=num_inference_steps,
            guidance_scale=3.5,
            generator=generator,
        ).images[0]
        image.save(f"./results/{file_name+ str(i)}.jpg")

In [ ]:
predict_CC_loop(text=["كيف حالك؟"],image_size=[512,512],file_name='How are you?')


a street sign in city, 'كيف حالك؟', filmfotos, film grain, reversal film photography


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
predict_CC_loop(text=["أهلاً وسهلاً"],image_size=[512,512],file_name='You are most welcome')
